# Travel Audience Data Science Challenge #

## Goal ##

One of the main problems we face at travel audience is identifying users that will eventually book a trip to an advertised destination. In this challenge, you are tasked to build a classifier to predict the conversion likelihood of a user based on previous search events, with emphasis on the feature engineering and evaluation part.

## Data ##

You are provided with two sample data sets

- `events.csv.gz` - A sample of events collected from an online travel agency, containing:
  * `ts` - the timestamp of the event
  * `event_type` - either `search` for searches made on the site, or `book` for a conversion, e.g. the user books the flight
  * `user_id` - unique identifier of a user
  * `date_from` - desired start date of the journey
  * `date_to` - desired end date of the journey
  * `origin` - IATA airport code of the origin airport
  * `destination` - IATA airport code of the destination airport
  * `num_adults` - number of adults
  * `num_children` - number of children

- `iata.csv` - containing geo-coordinates of major airports
  * `iata_code` - IATA code of the airport
  * `lat` - latitude in floating point format
  * `lon` - longitude in floating point format

## Tasks ##

Your code needs to do the following:

- Data preparation:
  - Calculate the geographic distance between origins and destinations
  - Convert raw data to a format suitable for the classification task
- Feature_engineering:
  - Based on the given input data, compute and justify three features of your choice that are relevant for predicting converters
- Experimental design:
  - Split data into test and training sets in a meaningful way
- Model:
  - A classifier of your choice that predicts the conversion-likelihood of a user

Use your best judgment to define rules and logic to compute each feature. Don't forget to comment your code!

## Deliverables ##

Code & comments that satisfy the tasks and demonstrate your coding style in Python or R. In addition, instructions on how to run your code.

We'll be evaluating the quality of your code, communication, and general solution design. We won't evaluate the actual performance of your model.

<br>
<br>

### Imports ###

In [84]:
import numpy as np
import pandas as pd
from numpy import cos, sin, arcsin, sqrt
from math import radians
from haversine import haversine, Unit

In [68]:
events = pd.read_csv('events.csv')

In [69]:
iata = pd.read_csv('iata.csv')

In [81]:
events.head()

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,dest_lat,dest_lon,lat_diff,lon_diff,travel_dist,trip_len
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,-85.864998,-73.578576,74.504691,91.281307,-160.369689,-164.859883,229.993952,6.0
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,-88.026756,-78.978109,0.443496,149.840853,-88.470251,-228.818962,245.326522,21.0
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,-85.879290,117.004643,86.505728,-128.116016,-172.385018,245.120659,299.667702,3.0
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,-85.879290,117.004643,27.961273,166.863410,-113.840563,-49.858767,124.280209,3.0
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,-37.742818,-111.586759,1.697775,30.674010,-39.440594,-142.260769,147.626850,6.0


In [71]:
events = events.merge(iata, how='left', left_on='origin', right_on='iata_code')

In [72]:
events.rename(columns={'lat': 'origin_lat', 'lon': 'origin_lon'}, inplace=True)
del events['iata_code']

In [73]:
events = events.merge(iata, how='left', left_on='destination', right_on='iata_code')

In [74]:
events.rename(columns={'lat': 'dest_lat', 'lon': 'dest_lon'}, inplace=True)
del events['iata_code']

In [75]:
events.columns

Index(['ts', 'event_type', 'user_id', 'date_from', 'date_to', 'origin',
       'destination', 'num_adults', 'num_children', 'origin_lat', 'origin_lon',
       'dest_lat', 'dest_lon'],
      dtype='object')

In [76]:
events['event_type'].value_counts()

search    45198
book       1809
Name: event_type, dtype: int64

In [77]:
events['lat_diff'] = events['origin_lat'] - events['dest_lat']

In [78]:
events['lon_diff'] = events['origin_lon'] - events['dest_lon']

In [79]:
events['travel_dist'] = np.sqrt((events['lat_diff'] ** 2) + (events['lon_diff'] ** 2))

In [80]:
events['trip_len'] = (pd.to_datetime(events['date_to']) - pd.to_datetime(events['date_from'])).dt.days

In [92]:
events['distance'] = events.apply(lambda x: haversine((x['origin_lat'], x['origin_lon']), (x['dest_lat'], x['dest_lon']), unit='km'), axis=1)

In [93]:
events

,ts,event_type,user_id,date_from,date_to,origin,destination,num_adults,num_children,origin_lat,origin_lon,dest_lat,dest_lon,lat_diff,lon_diff,travel_dist,trip_len,distance
0,2017-04-27 11:06:51,search,60225f,2017-06-01,2017-06-07,PAR,NYC,6,1,-85.864998,-73.578576,74.504691,91.281307,-160.369689,-164.859883,229.993952,6.0,18730.457064
1,2017-04-27 20:15:27,book,e5d69e,2017-08-12,2017-09-02,FRA,WAS,3,1,-88.026756,-78.978109,0.443496,149.840853,-88.470251,-228.818962,245.326522,21.0,10201.310759
2,2017-04-27 23:03:43,book,f953f0,2017-10-08,2017-10-11,BER,CGN,2,0,-85.879290,117.004643,86.505728,-128.116016,-172.385018,245.120659,299.667702,3.0,19555.872156
3,2017-04-27 15:17:50,book,794d35,2017-04-28,2017-05-01,BER,BCN,1,0,-85.879290,117.004643,27.961273,166.863410,-113.840563,-49.858767,124.280209,3.0,12816.477125
4,2017-04-27 22:51:57,book,ca4f94,2017-05-16,2017-05-22,DEL,BKK,4,0,-37.742818,-111.586759,1.697775,30.674010,-39.440594,-142.260769,147.626850,6.0,14458.812850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47002,2017-04-28 17:05:43,search,35479c,2017-04-30,2017-04-30,BER,CDG,1,0,-85.879290,117.004643,-4.611697,121.062616,-81.267593,-4.057973,81.368845,0.0,9037.713989
47003,2017-04-28 23:19:10,search,b72b1f,2017-05-02,2017-05-04,HAM,FRA,1,0,-65.797210,29.715909,-88.026756,-78.978109,22.229546,108.694018,110.943870,2.0,2768.896772
47004,2017-04-28 16:45:17,search,3390cd,2017-05-16,2017-05-21,STR,LON,2,2,-23.446015,-22.124003,46.840676,49.236225,-70.286690,-71.360228,100.162373,5.0,10579.564210
47005,2017-04-29 03:45:54,search,0b2be5,2017-05-04,2017-05-04,DUS,BRU,2,0,41.220876,147.690852,-67.590660,75.695470,108.811536,71.995382,130.473313,0.0,13495.803107
